In [1]:

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from data_pipeline import transformation_pipeline
from sklearn.model_selection import train_test_split
import tensorflow as tf
tf.random.set_seed(2111)

In [2]:

data = pd.read_csv('content/preprocessed_train.csv')

# let's pick the data wiht primary_use==0
data = data.query('primary_use==0 & meter==0')

In [3]:
b_id = [118, 646, 125]

In [4]:
def plot_output(actual, predicted, title, building_idx, avg_loss, avg_rmse):
    fig, (ax1, ax2, ax) = plt.subplots(3, 1,  figsize=(30, 15), sharex=True)

    fig.suptitle(
        f'{title} model for building {building_idx}\nmse={avg_loss:.5f}\nrmse={avg_rmse:.5f}', fontsize=24)

    ax1.plot(range(len(actual)),
             predicted,
             color='green', linestyle='dashed')
    ax1.set_title('Predicted')
    ax1.set_ylim(0, 1)

    ax2.plot(range(len(actual)),
             actual,
             color='red', label='Actual')
    ax2.set_title('Actual')
    ax2.set_ylim(0, 1)

    ax.plot(range(len(actual)),
            predicted,
            color='green', linestyle='dashed',
            label='Predicted')

    ax.plot(range(len(actual)),
            actual,
            color='red',
            label='actual')
    ax.set_ylim(0, 1)
    plt.legend()
    plt.show()
    fig.savefig(
        f'Final Plots for the research/tune on 20/{title} model -- building {building_idx} -- mse {avg_loss:.5f} -- rmse {avg_rmse:.5f}.png')

# %%

# function to laod certain building id


def loading_data(idx, train_size):
    pipeline, data_cleaned = transformation_pipeline(
        data, building_id=idx, meter=0, primary_use=0)

    train, test = train_test_split(data_cleaned,
                                   # [:, 1:],
                                   #transformed_data[:, 0],
                                   train_size=train_size,
                                   shuffle=False,
                                   random_state=2021)

    train_data = pipeline.fit_transform(train)
    test_data = pipeline.transform(test)

    x_train, y_train = train_data[:, 1:], train_data[:, 0]

    x_val, y_val = test_data[:, 1:], test_data[:, 0]

    train_gen = tf.keras.preprocessing.sequence.TimeseriesGenerator(x_train,
                                                                    y_train,
                                                            length=6, sampling_rate=1,
                                                             stride=1,
                                                             batch_size=32,
                                                                shuffle=False)

    val_gen = tf.keras.preprocessing.sequence.TimeseriesGenerator(x_val,
                                                                  y_val,
                                                      length=6, sampling_rate=1,
                                                      stride=1, batch_size=350,
                                                      shuffle=False
                                                      )
    return train_gen, val_gen[0]

In [5]:
pd.options.mode.chained_assignment = None
# %% loading models to evaluate them on new data
import time
models = ['models/Transformer_adam',
          'models/GRU_ADAM',
          'models/LSTM_ADAM',
          'models/RNN_ADAM']
finetuning_data = {}
for building_idx in b_id:
    train_gen, test_gen = loading_data(building_idx, 0.2)
    finetuning_data[building_idx] = {}
    for model_address in models:
        predicted = np.array([])
        actual = np.array([])

        txt = model_address.split('/')[1].split('_')[0]

        finetuning_data[building_idx][txt] = []

        model = tf.keras.models.load_model(model_address)

       # start_time = time.time()

        model.fit(train_gen, epochs=15, verbose=False)
       # time_taken = time.time() - start_time
       # print(f"model {txt} took {time.time() - start_time} seconds")

        predicted = np.append(predicted, model.predict(test_gen[0]))
        actual = np.append(actual, test_gen[1])

        avg_mse = np.mean((actual - predicted)**2)
        avg_rmse = np.sqrt(np.mean((actual - predicted)**2))

        #finetuning_data[building_idx][txt].append(
        #    (avg_mse, avg_rmse, time_taken))

        mape= np.mean(np.abs(np.divide((predicted - actual),(actual + 1e-15))))

        print(f'{txt} Model \nbuilding: {building_idx} \nmse= {avg_mse} \nrmse] {avg_rmse} \nmape= {mape}\n----------')

        #plot_output(actual, predicted, txt, building_idx, avg_mse, avg_rmse)

Transformer Model 
building: 118 
mse= 0.00936779126558001 
rmse] 0.0967873507519449 
mape= 0.19034668517437536
----------
GRU Model 
building: 118 
mse= 0.008136873175614814 
rmse] 0.09020461837187059 
mape= 0.15619631910260892
----------
LSTM Model 
building: 118 
mse= 0.015584076218718955 
rmse] 0.12483619754990519 
mape= 0.26039965990744396
----------
RNN Model 
building: 118 
mse= 0.048006197682977 
rmse] 0.2191031667570713 
mape= 0.4423555671118504
----------
Transformer Model 
building: 646 
mse= 0.035827769834770325 
rmse] 0.18928224912751412 
mape= 924770065717.572
----------
GRU Model 
building: 646 
mse= 0.017834408565941193 
rmse] 0.13354552993620264 
mape= 1026355624199.8346
----------
LSTM Model 
building: 646 
mse= 0.028449453736345803 
rmse] 0.16866965861216948 
mape= 1221226879529.9912
----------
RNN Model 
building: 646 
mse= 0.09469632810814715 
rmse] 0.30772768498811925 
mape= 295408602273.69604
----------
Transformer Model 
building: 125 
mse= 0.012138896823412931 

In [6]:
# transformer, gru, lstm,rnn = [], [], [],[]
# for building_idx in b_id:

#     x, y, z,r = finetuning_data[building_idx].values()
#     transformer.append(x)
#     gru.append(y)
#     lstm.append(z)
#     rnn.append(r)
# # %%
# mse, rmse, t = [], [], []
# for d in [transformer, gru, lstm,rnn]:
#     for i in range(3):

#         mse.append(d[i][0][0])
#         rmse.append(d[i][0][1])
#         t.append(d[i][0][2])

#     d.append([np.mean(mse), np.mean(rmse), np.mean(t)])

# # %%
# for d, n in zip([transformer, gru, lstm,rnn], ['transformer', 'gru',
#                                                'lstm','rnn']):
#     print(f'--- {n} ---')
#     print(' mse \t rmse \t time')
#     print(f'{d[3][0]:0.4f}\t{d[3][1]:0.4f}\t{d[3][2]:0.4f}')